In [61]:
# 사용할 모듈 불러오기 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import re

In [62]:
# 데이터 불러오기
items = pd.read_csv("./data/items.csv")
shops = pd.read_csv("./data/shops.csv")
train = pd.read_csv("./data/sales_train.csv")
test = pd.read_csv("./data/test.csv")
item_categories = pd.read_csv('./data/item_categories.csv')

### train에 존재하지 않는 test값을 예측하기 위해 item_category_id를 사용한다.
- item_category의 앞부분만 자르고 새롭게 만든 열을 라벨인코딩 (악세사리 >> 1)
- item_id와 연결되어있는 item_category를 새롭게 만든 열로 연결하여 test값을 예측한다면?

In [63]:
# item_category 분류하여 새로운 열 만들기
item_categories

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4
...,...,...
79,Служебные,79
80,Служебные - Билеты,80
81,Чистые носители (шпиль),81
82,Чистые носители (штучные),82


In [64]:
item_categories.item_category_name.str.split('-').str[0]
# https://pydole.tistory.com/entry/Python-split%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%98%EC%97%AC-%EC%BB%AC%EB%9F%BC-%EB%B6%84%EB%A6%AC%ED%95%98%EA%B8%B0

0                           PC 
1                   Аксессуары 
2                   Аксессуары 
3                   Аксессуары 
4                   Аксессуары 
                ...            
79                    Служебные
80                   Служебные 
81      Чистые носители (шпиль)
82    Чистые носители (штучные)
83             Элементы питания
Name: item_category_name, Length: 84, dtype: object

In [65]:
item_categories['seob'] = item_categories.item_category_name.str.split('-').str[0]
item_categories
#'-'으로 나눠야 할거같음..

,item_category_name,item_category_id,seob
0,PC - Гарнитуры/Наушники,0,PC
1,Аксессуары - PS2,1,Аксессуары
2,Аксессуары - PS3,2,Аксессуары
3,Аксессуары - PS4,3,Аксессуары
4,Аксессуары - PSP,4,Аксессуары
...,...,...,...
79,Служебные,79,Служебные
80,Служебные - Билеты,80,Служебные
81,Чистые носители (шпиль),81,Чистые носители (шпиль)
82,Чистые носители (штучные),82,Чистые носители (штучные)


In [76]:
item_categories.duplicated(['seob'], keep='first')
# https://pydole.tistory.com/entry/Python-pandas-%EC%A4%91%EB%B3%B5%EA%B0%92-%EC%B2%98%EB%A6%AC-duplicates-dropduplicates

0     False
1     False
2      True
3      True
4      True
      ...  
79    False
80    False
81    False
82    False
83    False
Length: 84, dtype: bool

In [67]:
item_categories_spt = item_categories.drop_duplicates(['seob'], keep='first')
item_categories_spt
# 중복치 제거

,item_category_name,item_category_id,seob
0,PC - Гарнитуры/Наушники,0,PC
1,Аксессуары - PS2,1,Аксессуары
8,Билеты (Цифра),8,Билеты (Цифра)
9,Доставка товара,9,Доставка товара
10,Игровые консоли - PS2,10,Игровые консоли
18,Игры - PS2,18,Игры
26,Игры Android - Цифра,26,Игры Android
27,Игры MAC - Цифра,27,Игры MAC
28,Игры PC - Дополнительные издания,28,Игры PC
32,"Карты оплаты (Кино, Музыка, Игры)",32,"Карты оплаты (Кино, Музыка, Игры)"


In [68]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
item_categories_spt['seob_le'] = le.fit_transform(item_categories_spt['seob'])
le.inverse_transform(item_categories_spt['seob_le'] )
# 라벨인코딩

/var/folders/pq/_bzx9rqx2hs53brb_m6pwqtc0000gn/T/ipykernel_22306/2131502951.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_categories_spt['seob_le'] = le.fit_transform(item_categories_spt['seob'])


array(['PC ', 'Аксессуары ', 'Билеты (Цифра)', 'Доставка товара',
       'Игровые консоли ', 'Игры ', 'Игры Android ', 'Игры MAC ',
       'Игры PC ', 'Карты оплаты (Кино, Музыка, Игры)', 'Карты оплаты ',
       'Кино ', 'Книги ', 'Музыка ', 'Подарки ', 'Программы ',
       'Служебные', 'Служебные ', 'Чистые носители (шпиль)',
       'Чистые носители (штучные)', 'Элементы питания'], dtype=object)

In [69]:
item_categories_spt

,item_category_name,item_category_id,seob,seob_le
0,PC - Гарнитуры/Наушники,0,PC,0
1,Аксессуары - PS2,1,Аксессуары,1
8,Билеты (Цифра),8,Билеты (Цифра),2
9,Доставка товара,9,Доставка товара,3
10,Игровые консоли - PS2,10,Игровые консоли,4
18,Игры - PS2,18,Игры,5
26,Игры Android - Цифра,26,Игры Android,6
27,Игры MAC - Цифра,27,Игры MAC,7
28,Игры PC - Дополнительные издания,28,Игры PC,8
32,"Карты оплаты (Кино, Музыка, Игры)",32,"Карты оплаты (Кино, Музыка, Игры)",10


In [70]:
item_categories['seob_LE'] = item_categories_spt['seob_le']
item_categories

,item_category_name,item_category_id,seob,seob_LE
0,PC - Гарнитуры/Наушники,0,PC,0.0
1,Аксессуары - PS2,1,Аксессуары,1.0
2,Аксессуары - PS3,2,Аксессуары,NaN
3,Аксессуары - PS4,3,Аксессуары,NaN
4,Аксессуары - PSP,4,Аксессуары,NaN
...,...,...,...,...
79,Служебные,79,Служебные,16.0
80,Служебные - Билеты,80,Служебные,17.0
81,Чистые носители (шпиль),81,Чистые носители (шпиль),18.0
82,Чистые носители (штучные),82,Чистые носители (штучные),19.0


In [75]:
item_categories = item_categories.fillna(method='ffill')
item_categories
# https://rfriend.tistory.com/262

,item_category_name,item_category_id,seob,seob_LE
0,PC - Гарнитуры/Наушники,0,PC,0.0
1,Аксессуары - PS2,1,Аксессуары,1.0
2,Аксессуары - PS3,2,Аксессуары,1.0
3,Аксессуары - PS4,3,Аксессуары,1.0
4,Аксессуары - PSP,4,Аксессуары,1.0
...,...,...,...,...
79,Служебные,79,Служебные,16.0
80,Служебные - Билеты,80,Служебные,17.0
81,Чистые носители (шпиль),81,Чистые носители (шпиль),18.0
82,Чистые носители (штучные),82,Чистые носители (штучные),19.0


In [77]:
item_categories = item_categories.astype({'seob_LE':'int64'})
item_categories
# 한번 이렇게 만들어봤답니다 :D

,item_category_name,item_category_id,seob,seob_LE
0,PC - Гарнитуры/Наушники,0,PC,0
1,Аксессуары - PS2,1,Аксессуары,1
2,Аксессуары - PS3,2,Аксессуары,1
3,Аксессуары - PS4,3,Аксессуары,1
4,Аксессуары - PSP,4,Аксессуары,1
...,...,...,...,...
79,Служебные,79,Служебные,16
80,Служебные - Билеты,80,Служебные,17
81,Чистые носители (шпиль),81,Чистые носители (шпиль),18
82,Чистые носители (штучные),82,Чистые носители (штучные),19
